In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("분류된_결과.xlsx")

df.head()

,정렬Key(업체_품명기준),정렬Key(MES품명기준),주문번호,마감일자,외주처명,업체_품명,생산량,주문일자,제품명,제대타입,품목너비,품목높이,단가key,업체단가,표준단가,차이,업체_공급가액,추정_공급가액,업체_품명기준_Case
0,24뉴트리션 150*220+40 스탠드 지퍼-삼방스탠드-지퍼-150-220,24뉴트리션 150*220+40 스탠드 지퍼-삼방스탠드-지퍼-150-220,20240913-1619P0121,45566,진포장,24뉴트리션 150*220+40 스탠드 지퍼,77350,20240913,패트100,삼방스탠드-지퍼,150.0,220.0,삼방스탠드-지퍼-패트100-150,21.0,21.0,0.0,1624350.0,1624350.0,Case0
1,3M-2223H 210*210 삼방-삼방-210-210,3M-2223H 210*210 삼방-삼방-210-210,20240805-13160016,45536,진포장,3M-2223H 210*210 삼방,21800,20240805,은박90,삼방,210.0,210.0,삼방-은박90-210,6.0,6.0,0.0,130800.0,130800.0,Case0
2,3M-2223H 도로명주소 210*210 삼방-삼방-210-210,3M-2223H 도로명주소 210*210 삼방-삼방-210-210,20240805-13160016,45536,진포장,3M-2223H 도로명주소 210*210 삼방,81770,20240805,은박90,삼방,210.0,210.0,삼방-은박90-210,6.0,6.0,0.0,490620.0,490620.0,Case0
3,4대가이어가는한씨떡집 250*350 삼방-삼방-250-350,4대가이어가는한씨떡집 250*350 삼방-삼방-250-350,20241011-1000137720,45566,진포장,4대가이어가는한씨떡집 250*350 삼방,20550,20241011,나이론70,삼방,250.0,350.0,삼방-나이론70-250,7.0,7.0,0.0,143850.0,143850.0,Case0
4,DBR동아일보사 250*290+50 삼방 풀발이-2단삼방-지퍼-250-290,DBR동아일보사 250*290+50 삼방 풀발이-2단삼방-지퍼-250-290,20240910-1000135557,45566,진포장,DBR동아일보사 250*290+50 삼방 풀발이,22900,20240910,패씨피피80,2단삼방-지퍼,250.0,290.0,2단삼방-지퍼-패씨피피80-250,20.0,25.0,5.0,458000.0,572500.0,Case1


In [4]:
Case1 = df[df["업체_품명기준_Case"] == "Case1"]
Case2 = df[df["업체_품명기준_Case"] == "Case2"]

df_a = pd.concat([Case1, Case2], axis=0)
df_a.shape

(213, 19)

In [5]:
Case1.shape, Case2.shape

((203, 19), (10, 19))

In [7]:
df_a = df_a.reset_index()

In [9]:
df_a.drop("index", axis=1, inplace=True)

In [45]:
# 각 정렬Key업체_품명기준)별로 업체단가의 Max, Min, 표준단가를 구한다.

na = df_a.groupby(
    "정렬Key(업체_품명기준)"
).agg(
    업체단가_MAX = ('업체단가', 'max'),
    업체단가_MIN=('업체단가', 'min'),
    업체단가_AVG = ('업체단가', 'mean')
).reset_index()

In [46]:
na.shape

(152, 4)

In [47]:
na.columns

Index(['정렬Key(업체_품명기준)', '업체단가_MAX', '업체단가_MIN', '업체단가_AVG'], dtype='object')

In [30]:
df_a.shape

(213, 19)

In [49]:
# 그룹화된 데이터프레임과 기존 데이터프레임을 merge한다.

df_merged = pd.merge(df_a, na, on='정렬Key(업체_품명기준)', how='left')

# 오차범위 계산 함수 (수정)
def is_within_10_percent(row):
    standard_price = row['표준단가']  # 원본 '표준단가' 사용
    max_price = row['업체단가_MAX']
    min_price = row['업체단가_MIN']
    
    max_diff = abs(max_price - standard_price) / standard_price
    min_diff = abs(min_price - standard_price) / standard_price
    
    max_within_10 = 'O' if max_diff <= 0.1 else 'X'
    min_within_10 = 'O' if min_diff <= 0.1 else 'X'
    
    return pd.Series([max_within_10, min_within_10], index=['Max_10%_오차범위', 'Min_10%_오차범위'])

# 오차범위 결과를 df_merged에 추가
df_merged[['Max_10%_오차범위', 'Min_10%_오차범위']] = df_merged.apply(is_within_10_percent, axis=1)

# 결과 출력
#print(df_merged[['정렬Key(업체_품명기준)', '업체단가', '업체단가_Max', '업체단가_Min', '표준단가', 'Max_10%_오차범위', 'Min_10%_오차범위']])

    

In [50]:
df_merged.head()

,정렬Key(업체_품명기준),정렬Key(MES품명기준),주문번호,마감일자,외주처명,업체_품명,생산량,주문일자,제품명,제대타입,...,표준단가,차이,업체_공급가액,추정_공급가액,업체_품명기준_Case,업체단가_MAX,업체단가_MIN,업체단가_AVG,Max_10%_오차범위,Min_10%_오차범위
0,DBR동아일보사 250*290+50 삼방 풀발이-2단삼방-지퍼-250-290,DBR동아일보사 250*290+50 삼방 풀발이-2단삼방-지퍼-250-290,20240910-1000135557,45566,진포장,DBR동아일보사 250*290+50 삼방 풀발이,22900,20240910,패씨피피80,2단삼방-지퍼,...,25.0,5.0,458000.0,572500.0,Case1,20.0,20.0,20.0,X,X
1,NT네이쳐티공용파우치 210*278+50 스탠드 지퍼 오픈-삼방스탠드-지퍼-210-280,NT네이쳐티공용파우치 210*278+50 스탠드 지퍼 오픈-삼방스탠드-지퍼-210-280,20240807-1000163536,45536,진포장,NT네이쳐티공용파우치 210*278+50 스탠드 지퍼 오픈,25900,20240807,은박(무광)130,삼방스탠드-지퍼,...,29.0,0.6,766640.0,751100.0,Case1,29.6,29.6,29.6,O,O
2,TOH 지퍼백 185*245 삼방 지퍼-삼방-지퍼-185-245,TOH 지퍼백 185*245 삼방 지퍼-삼방-지퍼-185-245,20240822-1000168835,45536,진포장,TOH 지퍼백 185*245 삼방 지퍼,30990,20240822,오씨피피(투명)80,삼방-지퍼,...,18.0,0.5,573315.0,557820.0,Case1,18.5,18.5,18.5,O,O
3,TR 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,TR 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,20240912-1000168934,45566,진포장,TR 통단팥죽 420*320+45 삼방 스탠드,13220,20240912,패트(유백/유무광)82,삼방스탠드,...,42.0,21.0,832860.0,555240.0,Case1,63.0,63.0,63.0,X,X
4,TR) 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,TR) 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,20240830-1000168934,45536,진포장,TR) 통단팥죽 420*320+45 삼방 스탠드,13060,20240830,패트(유백/유무광)82,삼방스탠드,...,42.0,21.0,822780.0,548520.0,Case1,63.0,63.0,63.0,X,X


In [51]:
df_merged.shape

(213, 24)

In [52]:
df_merged.head()

,정렬Key(업체_품명기준),정렬Key(MES품명기준),주문번호,마감일자,외주처명,업체_품명,생산량,주문일자,제품명,제대타입,...,표준단가,차이,업체_공급가액,추정_공급가액,업체_품명기준_Case,업체단가_MAX,업체단가_MIN,업체단가_AVG,Max_10%_오차범위,Min_10%_오차범위
0,DBR동아일보사 250*290+50 삼방 풀발이-2단삼방-지퍼-250-290,DBR동아일보사 250*290+50 삼방 풀발이-2단삼방-지퍼-250-290,20240910-1000135557,45566,진포장,DBR동아일보사 250*290+50 삼방 풀발이,22900,20240910,패씨피피80,2단삼방-지퍼,...,25.0,5.0,458000.0,572500.0,Case1,20.0,20.0,20.0,X,X
1,NT네이쳐티공용파우치 210*278+50 스탠드 지퍼 오픈-삼방스탠드-지퍼-210-280,NT네이쳐티공용파우치 210*278+50 스탠드 지퍼 오픈-삼방스탠드-지퍼-210-280,20240807-1000163536,45536,진포장,NT네이쳐티공용파우치 210*278+50 스탠드 지퍼 오픈,25900,20240807,은박(무광)130,삼방스탠드-지퍼,...,29.0,0.6,766640.0,751100.0,Case1,29.6,29.6,29.6,O,O
2,TOH 지퍼백 185*245 삼방 지퍼-삼방-지퍼-185-245,TOH 지퍼백 185*245 삼방 지퍼-삼방-지퍼-185-245,20240822-1000168835,45536,진포장,TOH 지퍼백 185*245 삼방 지퍼,30990,20240822,오씨피피(투명)80,삼방-지퍼,...,18.0,0.5,573315.0,557820.0,Case1,18.5,18.5,18.5,O,O
3,TR 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,TR 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,20240912-1000168934,45566,진포장,TR 통단팥죽 420*320+45 삼방 스탠드,13220,20240912,패트(유백/유무광)82,삼방스탠드,...,42.0,21.0,832860.0,555240.0,Case1,63.0,63.0,63.0,X,X
4,TR) 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,TR) 통단팥죽 420*320+45 삼방 스탠드-삼방스탠드-420-320,20240830-1000168934,45536,진포장,TR) 통단팥죽 420*320+45 삼방 스탠드,13060,20240830,패트(유백/유무광)82,삼방스탠드,...,42.0,21.0,822780.0,548520.0,Case1,63.0,63.0,63.0,X,X


In [41]:
df_merged.to_excel("단가_오차범위_계산.xlsx", index=False)

In [ ]:
filepath = "C:\\Users\\minsu\\OneDrive\\Desktop\\파일_관리\\협력사(제대)매출추정_대시보드_류홍규\\10월_마감내역서_추정공급가액_비교"

with pd.ExcelWriter(filepath) as writer:
    df_sorted_업체.to_excel(writer, sheet_name='업체_품명기준_분류', index=False)
    